## Session 数据管理
FlaskRuntime和ChainlitRuntime 提供 Session 数据的管理功能，允许跟踪和存储用户会话数据。一般只有在二次开发的组件需要使用该能力。

**1、 二次开发组件**

二次开发的组件需要重写组件的 run(message, stream, **args)方法，并且至少需要有 message 和 stream 两个参数。

下面基于 QueryRewrite 和 Playground 两个组件，开发 PlaygroundWithHistory 组件，该组件需要对会话数据进行操作。

当使用 Component 独立运行时，会话数据会被存储于内存。


In [ ]:
import os
import logging
from appbuilder.core.component import Component
from appbuilder import (
    UserSession, Message, QueryRewrite, Playground,
)

# 使用组件之前，请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥
os.environ["APPBUILDER_TOKEN"] = '...'

class PlaygroundWithHistory(Component):
    def __init__(self):
        super().__init__()
        self.query_rewrite = QueryRewrite(model="Qianfan-Agent-Speed-8K")
        self.playground = Playground(
            prompt_template="{query}",
            model="ERNIE-Bot"
        )

    def run(self, message: Message, stream: bool=False):
        user_session = UserSession()
        # 获取 Session 历史数据
        history_queries = user_session.get_history("query", limit=1)
        history_answers = user_session.get_history("answer", limit=1)

        # query 改写
        if history_queries and history_answers:
            history = []
            for query, answer in zip(history_queries, history_answers):
                history.extend([query.content, answer.content])
            logging.info(f"history: {history}")
            message = self.query_rewrite(
                Message(history + [message.content]), rewrite_type="带机器人回复")
        logging.info(f"message: {message}") 

        # 执行 playground
        answer = self.playground.run(message, stream)

        # 保存本轮数据
        user_session.append({
            "query": message,
            "answer": answer,
        }) 
        return answer

# component 可以独立运行，session数据会被保存于内存
playground_with_history_component = PlaygroundWithHistory()
print(playground_with_history_component.run(Message("海淀区的面积是多少"), stream=False))

**2、 会话数据存储数据库**

使用 FlaskRuntime和ChainlitRuntime 对 Component 服务化，会话数据会被存储于数据库。
下面的代码以 SQLite 为例展示该能力，更多数据库配置详见[文档](https://docs.sqlalchemy.org/en/20/core/engines.html#backend-specific-urls)

In [ ]:
import appbuilder
from appbuilder.utils.flask_deploy import FlaskRuntime
user_session_config = "sqlite:///foo.db"
agent = FlaskRuntime(
    component=playground_with_history_component, 
    user_session_config=user_session_config)
agent.serve(port=8091)

**3、 查看user_session.db储存信息**


使用该能力，查看用户对话信息。

In [83]:
import sqlite3  
  
# 连接到 SQLite 数据库  
# 如果文件不存在，会自动在当前目录创建:  
user_session_path = '本地user_session.db地址' 
conn = sqlite3.connect(user_session_path)  
cursor = conn.cursor()  

执行 SQL 语句，列出SQLite数据库中的所有表

In [85]:
# 执行一条 SQL 语句，列出所有表  
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")  
print(cursor.fetchall())  

[('appbuilder_session_messages',)]


查询appbuilder_session_messages表的列信息,appbuilder_session_messages表的列信息为:
- id
- session_id
- request_id
- message_key
- message_value
- created_at
- updated_at
- deleted

In [79]:
cursor.execute("PRAGMA table_info(appbuilder_session_messages);")  
columns_info = cursor.fetchall()  

column_names = [info[1] for info in columns_info]  # info[1]是列名的位置  
for column_name in column_names:  
    print(column_name)   

id
session_id
request_id
message_key
message_value
created_at
updated_at
deleted


查询表中的特定数据【以message_value信息为例】
输出content、id、token_usage等信息

In [80]:
import json
cursor.execute("SELECT message_value FROM appbuilder_session_messages;")  
for row in cursor.fetchall():  
    print(json.loads(row[0]))

{'content': '你好', 'name': 'msg', 'mtype': 'str', 'id': '90c8d150-c7b9-44a4-ac77-50dd61ee329a'}
{'content': '你好，我是百度研发的知识增强大语言模型，中文名是文心一言，英文名是ERNIE Bot。我能够与人对话互动，回答问题，协助创作，高效便捷地帮助人们获取信息、知识和灵感。', 'name': 'msg', 'mtype': 'dict', 'id': '0fc6c5f7-23bf-4f25-8555-69b4155908b7', 'extra': {}, 'token_usage': {'prompt_tokens': 2, 'completion_tokens': 42, 'total_tokens': 44}}
{'content': '请介绍一下你自己', 'name': 'msg', 'mtype': 'dict', 'id': '4f5cf6d1-976e-456e-a632-93c4b2550523', 'extra': {'search_db': [{'content': '带机器人回复：请你扮演一个智能搜索改写补全机器人，请根据User的搜索历史以及对应的搜索结果，对最后一句话先进行主语继承改写，然后进行上下文信息补全，注意：不要改变原文的意思，答案要尽可能简洁，不要直接回答该问题，不要输出多于的内容。\\n\\n例子：\\n搜索历史：\\nUser：今天上午你干嘛了\\nAssistant：去打篮球啦\\nUser：好玩吗？\\n答案：\\n打篮球好玩吗？', 'dataset_id': '1f777fa3-26db-4237-98d5-075abc07a84f', 'dataset_name': '多轮改写Prompt', 'document_id': '92e0f30d-1f30-46f0-8377-59ab0fb6eb93', 'document_name': '多轮改写prompt_带机器人回复.txt', 'id': '61657591-f610-44f0-84ec-67cedcb66447', 'mock_id': '1', 'position': 0, 'score': 0.511053, 'sentences': [{'co

以id查询相关的agent_runtime信息

In [87]:
cursor.execute("SELECT * FROM appbuilder_session_messages WHERE session_id = 'b2c9d058-4475-4258-ad90-4334f3d024d5';")  
for tuple in cursor.fetchall():
    for message in tuple:
        try: 
            message = json.loads(message)
            print(message)
        except:
            print(message)

16010b88-d766-4524-81ee-37f96ceadb4d
b2c9d058-4475-4258-ad90-4334f3d024d5
0bdafb3d-f7e3-4187-bc1d-63cf51fbda29
query
{'content': '你好', 'name': 'msg', 'mtype': 'str', 'id': '90c8d150-c7b9-44a4-ac77-50dd61ee329a'}
2024-07-30 15:01:10.949475
2024-07-30 15:01:10.949485
0
4bd85fd0-2e5c-4de1-a47e-ea122f7e928c
b2c9d058-4475-4258-ad90-4334f3d024d5
0bdafb3d-f7e3-4187-bc1d-63cf51fbda29
answer
{'content': '你好，我是百度研发的知识增强大语言模型，中文名是文心一言，英文名是ERNIE Bot。我能够与人对话互动，回答问题，协助创作，高效便捷地帮助人们获取信息、知识和灵感。', 'name': 'msg', 'mtype': 'dict', 'id': '0fc6c5f7-23bf-4f25-8555-69b4155908b7', 'extra': {}, 'token_usage': {'prompt_tokens': 2, 'completion_tokens': 42, 'total_tokens': 44}}
2024-07-30 15:01:10.955875
2024-07-30 15:01:10.955884
0
d6e433fc-ef4d-4ade-9a63-fec600e95481
b2c9d058-4475-4258-ad90-4334f3d024d5
6300b76b-2307-4ed3-9f6d-e61dfc621ff8
query
{'content': '请介绍一下你自己', 'name': 'msg', 'mtype': 'dict', 'id': '4f5cf6d1-976e-456e-a632-93c4b2550523', 'extra': {'search_db': [{'content': '带机器人回复：请你扮演一个智能搜索改写补全机器人，请根据Us